In [1]:
!pip install torchsummaryX

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from torchsummaryX import summary
from torch.optim.lr_scheduler import StepLR

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="kkRSmrVtd2kYTs5QO1pL")
project = rf.workspace("shubham-karn").project("spot-i35zh")
dataset = project.version(4).download("folder")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to spot-4 in folder:: 100%|██████████| 4319/4319 [00:00<00:00, 5508.51it/s]


In [4]:
data_dir = '/content/spot-4/train'

In [5]:
torch.manual_seed(123)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
dataset = datasets.ImageFolder(data_dir, transform=transform)

In [9]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [10]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


In [11]:
model = models.resnet50(pretrained=False)

num_classes = 2
model.fc = nn.Sequential(
    nn.Flatten(),
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, num_classes),
    nn.Softmax(dim=1)
)

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.


In [12]:
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

In [13]:
model = model.cuda()

# Display the model summary
summary(model, torch.zeros((1, 3, 224, 224)).cuda())

                                             Kernel Shape       Output Shape  \
Layer                                                                          
0_conv1                                     [3, 64, 7, 7]  [1, 64, 112, 112]   
1_bn1                                                [64]  [1, 64, 112, 112]   
2_relu                                                  -  [1, 64, 112, 112]   
3_maxpool                                               -    [1, 64, 56, 56]   
4_layer1.0.Conv2d_conv1                    [64, 64, 1, 1]    [1, 64, 56, 56]   
5_layer1.0.BatchNorm2d_bn1                           [64]    [1, 64, 56, 56]   
6_layer1.0.ReLU_relu                                    -    [1, 64, 56, 56]   
7_layer1.0.Conv2d_conv2                    [64, 64, 3, 3]    [1, 64, 56, 56]   
8_layer1.0.BatchNorm2d_bn2                           [64]    [1, 64, 56, 56]   
9_layer1.0.ReLU_relu                                    -    [1, 64, 56, 56]   
10_layer1.0.Conv2d_conv3                

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_conv1,"[3, 64, 7, 7]","[1, 64, 112, 112]",9408.0,118013952.0
1_bn1,[64],"[1, 64, 112, 112]",128.0,64.0
2_relu,-,"[1, 64, 112, 112]",NaN,NaN
3_maxpool,-,"[1, 64, 56, 56]",NaN,NaN
4_layer1.0.Conv2d_conv1,"[64, 64, 1, 1]","[1, 64, 56, 56]",4096.0,12845056.0
...,...,...,...,...
157_fc.Flatten_0,-,"[1, 2048]",NaN,NaN
158_fc.Linear_1,"[2048, 512]","[1, 512]",1049088.0,1048576.0
159_fc.ReLU_2,-,"[1, 512]",NaN,NaN


In [14]:
print(len(train_dataset))
print(len(val_dataset))

3451
863


In [15]:
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0  # To accumulate the loss for the epoch

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total

    scheduler.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_loss:.4f}, Validation Accuracy: {accuracy:.4f}')

# After training is complete, you can print the final results
print(f'Training completed. Final Training Loss: {average_loss:.4f}, Final Validation Accuracy: {accuracy:.4f}')


# Save the trained model
torch.save(model.state_dict(), 'apple_spot_classifier.pth')

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


Epoch [1/150], Training Loss: 0.5168, Validation Accuracy: 0.7231
Epoch [2/150], Training Loss: 0.5049, Validation Accuracy: 0.8239
Epoch [3/150], Training Loss: 0.4834, Validation Accuracy: 0.8250
Epoch [4/150], Training Loss: 0.4644, Validation Accuracy: 0.8088
Epoch [5/150], Training Loss: 0.4383, Validation Accuracy: 0.8389
Epoch [6/150], Training Loss: 0.4152, Validation Accuracy: 0.9328
Epoch [7/150], Training Loss: 0.4363, Validation Accuracy: 0.8969
Epoch [8/150], Training Loss: 0.4022, Validation Accuracy: 0.9305
Epoch [9/150], Training Loss: 0.4022, Validation Accuracy: 0.9316
Epoch [10/150], Training Loss: 0.4000, Validation Accuracy: 0.9189
Epoch [11/150], Training Loss: 0.4046, Validation Accuracy: 0.8633
Epoch [12/150], Training Loss: 0.4154, Validation Accuracy: 0.9200
Epoch [13/150], Training Loss: 0.4297, Validation Accuracy: 0.8702
Epoch [14/150], Training Loss: 0.3975, Validation Accuracy: 0.9015
Epoch [15/150], Training Loss: 0.4093, Validation Accuracy: 0.8957
Epoc

In [19]:
import shutil

def copy_file_to_directory(source_file, destination_directory):
    try:
        # Copy the file to the destination directory
        shutil.copy(source_file, destination_directory)
        print(f"File '{source_file}' copied to '{destination_directory}' successfully.")
    except FileNotFoundError:
        print(f"Error: File '{source_file}' not found.")
    except IsADirectoryError:
        print(f"Error: '{destination_directory}' is a directory. Please provide a valid destination path.")
    except PermissionError:
        print(f"Error: Permission denied. Make sure you have the necessary permissions.")

# Example usage:
source_file = "apple_spot_classifier.pth"
destination_directory = "/content/drive/MyDrive/train"
copy_file_to_directory(source_file, destination_directory)


File 'apple_spot_classifier.pth' copied to '/content/drive/MyDrive/train' successfully.


In [17]:
model = torch.load("/content/apple_spot_classifier.pth")
print(model)

OrderedDict([('conv1.weight', tensor([[[[ 4.2286e-02, -2.0189e-02, -3.5668e-02,  ..., -3.5938e-03,
            1.7894e-02, -3.0491e-02],
          [-2.6621e-02,  6.8563e-03, -9.7517e-02,  ..., -2.2353e-02,
           -4.0258e-02, -1.6825e-02],
          [ 7.0628e-03,  6.6973e-03, -3.3865e-03,  ..., -2.3802e-02,
           -9.4246e-03, -1.0307e-02],
          ...,
          [ 4.2637e-03,  1.9239e-02,  3.5034e-02,  ..., -2.6589e-02,
           -2.8469e-02, -5.9892e-02],
          [ 2.8858e-02, -1.7513e-02, -6.5773e-03,  ..., -1.3502e-02,
            1.9894e-02, -4.9086e-02],
          [ 2.1459e-03,  4.3758e-02,  3.7811e-02,  ..., -2.1607e-02,
           -1.2600e-02, -4.3278e-02]],

         [[ 9.6897e-03,  6.8760e-03,  1.7734e-02,  ...,  2.8659e-02,
           -1.6695e-02, -2.1623e-02],
          [ 4.1211e-02,  3.8803e-03,  6.0343e-02,  ...,  2.7106e-02,
            2.8069e-02,  2.3179e-02],
          [-5.2657e-02, -3.1563e-02,  3.5681e-02,  ...,  5.5778e-02,
            1.7877e-02,  1.8

In [21]:
import torch
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn

# Load the pre-trained ResNet18 model
model = models.resnet50(pretrained=False)

# Modify the fully connected layer for your specific classification task
num_classes = 2
model.fc = nn.Sequential(
    nn.Flatten(),
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, num_classes),
    nn.Softmax(dim=1)
)

# Load the saved model state dictionary
model.load_state_dict(torch.load('/content/apple_spot_classifier.pth'))
model.eval()

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess the input image
image_path = '/content/WhatsApp Image 2024-01-13 at 11.01.06 PM (1).jpeg'
image = Image.open(image_path).convert('RGB')
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

# Make prediction
with torch.no_grad():
    model.eval()
    output = model(input_batch)
    _, predicted_class = torch.max(output, 1)

print(f'Predicted class: {predicted_class.item()}')


Predicted class: 0
